In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# base_path = '/content/drive/MyDrive/NLP/'
base_path = ''

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

model_name = "cambridgeltl/SapBERT-from-PubMedBERT-fulltext"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def get_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True,max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :]  # CLS token

In [ ]:
import pandas as pd
df = pd.read_csv(base_path + 'dataset/POLG_HPO.csv', encoding='utf-8')
embedding_List = []

In [ ]:
for i in df['name']:
    embedding_List.append(get_embedding(i))
    print(i)

 3-Methylglutaconic aciduria
 Abdominal cramps
 Abdominal distention
 Abdominal pain
 Abnormal cell morphology
 Abnormal cerebellar cortex morphology
 Abnormal cerebral white matter morphology
 Abnormal cerebrospinal fluid morphology
 Abnormal retinal morphology
 Abnormal thalamic MRI signal intensity
 Abnormal vestibular function
 Abnormality of central motor conduction
 Abnormality of extrapyramidal motor function
 Abnormality of eye movement
 Abnormality of movement
 Abnormality of the extraocular muscles
 Abnormality of the eye
 Abnormality of the gastrointestinal tract
 Abnormality of the hand
 Abnormality of the liver
 Abnormality of the mitochondrion
 Abnormality of vision
 Abnormality of visual evoked potentials
 Absent Achilles reflex
 Aciduria
 Action tremor
 Acute hepatic failure
 Adult onset
 Allodynia
 Anasarca
 Anemia
 Anxiety
 Areflexia
 Areflexia of lower limbs
 Arrhythmia
 Ascites
 Astrocytosis
 Ataxia
 Atrial fibrillation
 Atrophic muscularis propria
 Atrophy/Degenera

In [ ]:
def get_hpo_id(hpo_entity):
    hpo_entity_emb = get_embedding(hpo_entity)
    result_idx = 0
    max_sim = 0
    for index, item in enumerate(embedding_List):
        sim = F.cosine_similarity(hpo_entity_emb, item, dim=1)
        if sim > max_sim:
            max_sim = sim
            result_idx = index

    result_id = df['id'][result_idx]
    result_name = df['name'][result_idx]
    print(result_id, end=' ')
    print("---", end=' ')
    print(result_name)

In [ ]:
get_hpo_id('action tremor')

HP:0002345  ---  Action tremor


In [ ]:
import json
filepath = base_path + 'dataset/cleaned/NER/processed_merged.jsonl'
dl = []
hpo_list = []

with open(filepath, 'r', encoding='utf-8') as file:
    for line in file:
        data = json.loads(line.strip())
        dl.append(data)
        case = data
        for i in case['spans']:
            if i['label'] == 'HPO_TERM':
                hpo_list.append(case['text'][i['start']:i['end']])

In [ ]:
get_hpo_id(hpo_list[1])
for i in hpo_list:
    print(i)
    get_hpo_id(i)
    print("\n")

HP:0000739  ---  Anxiety
migraine headaches
HP:0002076  ---  Migraine


anxiety.
HP:0000739  ---  Anxiety


dystonic toe curling,
HP:0009027  ---  Foot dorsiflexor weakness


action tremor
HP:0002345  ---  Action tremor


reduced arm swing
HP:0002522  ---  Areflexia of lower limbs


hyporeflexia
HP:0001265  ---  Hyporeflexia


foot numbness,
HP:0009027  ---  Foot dorsiflexor weakness


distal weakness
HP:0002460  ---  Distal muscle weakness


axonal predominantly sensory neuropathy.
HP:0003390  ---  Sensory axonal neuropathy


facial masking
HP:0000298  ---  Mask-like facies


hypophonia
HP:0002791  ---  Hypoventilation


cogwheeling
HP:0002396  ---  Cogwheel rigidity


stooping
HP:0025403  ---  Stooped posture


shuffling gait
HP:0002362  ---  Shuffling gait


bradykinesia
HP:0002067  ---  Bradykinesia


lactate was elevated
HP:0002151  ---  Increased circulating lactate concentration


dystonic toe curling
HP:0009027  ---  Foot dorsiflexor weakness


action tremor
HP:0002345  ---  Ac